In [1]:
import argparse
import sys

import pyhealth

from pyhealth.datasets import MIMIC4Dataset, MIMIC3Dataset
from pyhealth.tasks import drug_recommendation_mimic4_fn, drug_recommendation_mimic3_fn
# import dataloader related functions
from pyhealth.datasets.splitter import split_by_patient
from pyhealth.datasets import split_by_patient, get_dataloader
# import gamenet model
from pyhealth.models import RETAIN, GAMENet
# import trainer
from pyhealth.trainer import Trainer


/home/ram/code/school/dl4h/final/submit/GAMENet/pyhealth/.v2/lib/python3.8/site-packages/pyhealth/trainer.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [2]:
from gamenet import *
from drug_rec_task import drug_recommendation_mimic4_no_hist

In [3]:
#data = prepare_drug_task_data(MIMIC4)
mimic_data = load_mimic_data(MIMIC4)

reading mimic4 data...
stat

Statistics of base dataset (dev=False):
	- Dataset: MIMIC4Dataset
	- Number of patients: 180733
	- Number of visits: 431231
	- Number of visits per patient: 2.3860
	- Number of events per visit in diagnoses_icd: 11.0296
	- Number of events per visit in procedures_icd: 1.5518
	- Number of events per visit in prescriptions: 54.2354

info

dataset.patients: patient_id -> <Patient>

<Patient>
    - visits: visit_id -> <Visit> 
    - other patient-level info
    
    <Visit>
        - event_list_dict: table_name -> List[Event]
        - other visit-level info
    
        <Event>
            - code: str
            - other event-level info



In [4]:
data = prepare_drug_task_data(mimic_data, MIMIC4)

drug task prepare


Generating samples for drug_recommendation_mimic4_fn: 100%|███████████████████████████████████| 180733/180733 [01:01<00:00, 2925.25it/s]


{'visit_id': '22595853', 'patient_id': '10000032', 'conditions': [['5723', '78959', '5715', '07070', '496', '29680', '30981', 'V1582']], 'procedures': [['5491']], 'drugs': ['B01A', 'J07B', 'A12B', 'C03D', 'C03C', 'N02B', 'J05A', 'R03A', 'N07B', 'R03B'], 'drugs_all': [['B01A', 'J07B', 'A12B', 'C03D', 'C03C', 'N02B', 'J05A', 'R03A', 'N07B', 'R03B']]}


In [5]:
nh_data = 

ImportError: cannot import name 'drug_recommendation_mimic4_no_hist' from 'drug_rec_task' (/home/ram/code/school/dl4h/final/submit/GAMENet/pyhealth/drug_rec_task.py)